In [30]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_columns = 100

In [31]:
df=pd.read_json("parsed_df.json")

In [32]:
dftouse=df.copy()
dftouse.head()

,ACTION_BEFORE,ACTION_TYPE,AWAY_PLAYER1_ID,AWAY_PLAYER2_ID,AWAY_PLAYER3_ID,AWAY_PLAYER4_ID,AWAY_PLAYER5_ID,AWAY_TEAM_ID,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,DRIBBLES,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,FINAL_MARGIN,GAME_EVENT_ID,GAME_ID,HOME_PLAYER1_ID,HOME_PLAYER2_ID,HOME_PLAYER3_ID,HOME_PLAYER4_ID,HOME_PLAYER5_ID,HOME_TEAM_ID,LOCATION,LOC_X,LOC_Y,PERIOD_x,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER2_ID,PLAYER3_ID,PLAYER_ID,PTS,PTS_TYPE,SCOREMARGIN,SEC_LEFT_G,SEC_LEFT_Q,SHOT_CLOCK,SHOT_DIST,SHOT_DISTANCE,SHOT_MADE_FLAG,SHOT_NUMBER,SHOT_RESULT,SHOT_ZONE_AREA,SHOT_ZONE_BASIC,SHOT_ZONE_RANGE,TEAM_ID,TOUCH_TIME,W,WCTIMESTRING,def_height,off_height
0,10,Jump Shot,2749,101131,201167,202696,203090,1610612753,101131,1.6,2,1,2,2,10,2,21300001,2561,201579,201588,202331,202362,1610612754,1,-38,45,1,4,0,0,0,0,2561,0,2,0,2859,699,6.7,7.4,5,0,1,missed,Center(C),In The Paint (Non-RA),Less Than 8 ft.,1610612754,7.8,True,7:12 PM,6-7,6-9
1,6,Pullup Jump shot,2749,101131,201167,202696,203090,1610612753,203090,4.1,1,79,1,5,10,5,21300001,2561,201579,201588,202331,202362,1610612754,1,105,164,1,4,0,0,0,0,202331,2,2,2,2850,690,16.8,19.3,19,1,1,made,Right Side Center(RC),Mid-Range,16-24 ft.,1610612754,3.2,True,7:13 PM,6-9,6-9
10,2,Tip Shot,2749,101131,201167,202696,203090,1610612753,201167,0.4,0,4,2,26,10,26,21300001,2561,201579,201588,202331,202362,1610612754,1,13,9,1,4,0,0,0,0,201579,0,2,8,2735,575,19.6,2.1,1,0,1,missed,Center(C),Restricted Area,Less Than 8 ft.,1610612754,0.0,True,7:16 PM,6-5,7-2
100,5,Layup Shot,2749,101131,201167,202696,203506,1610612753,101131,0.6,0,5,1,291,10,291,21300001,2561,201579,201588,202331,202362,1610612754,1,1,0,3,4,4,0,2561,0,202362,2,2,3,1156,436,9.3,2.5,0,1,9,made,Center(C),Restricted Area,Less Than 8 ft.,1610612754,0.0,True,8:36 PM,6-7,6-5
1000,1,Jump Shot,2403,2772,202344,202322,203078,1610612764,203083,13.6,0,1,2,67,-11,67,21300007,1497,203484,2746,202328,203083,1610612765,0,-229,135,1,5,0,0,0,0,2772,0,3,-3,2362,202,24.0,20.2,26,0,6,missed,Left Side Center(LC),Above the Break 3,24+ ft.,1610612764,2.7,False,8:02 PM,6-11,6-8


In [27]:
#create train and test set
from sklearn.cross_validation import train_test_split
itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.7)
mask=np.ones(dftouse.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

In [38]:
shooter = [u'ACTION_BEFORE', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', u'DRIBBLES', u'GAME_ID', u'LOCATION', u'LOC_X', u'LOC_Y', u'PERIOD_x', u'PLAYER_ID', u'PTS_TYPE', u'SCOREMARGIN', u'SEC_LEFT_G', u'SEC_LEFT_Q',  u'SHOT_CLOCK', u'SHOT_DISTANCE', u'SHOT_NUMBER', u'TEAM_ID', u'TOUCH_TIME']
defender = [u'ACTION_BEFORE', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', u'GAME_ID', u'LOCATION', u'LOC_X', u'LOC_Y', u'PERIOD_x', u'PLAYER_ID', u'PTS_TYPE', u'SCOREMARGIN', u'SEC_LEFT_G', u'SEC_LEFT_Q',  u'SHOT_CLOCK', u'SHOT_DISTANCE', u'TEAM_ID']

In [40]:
from sklearn.svm import LinearSVC
clfsvm=LinearSVC(loss="hinge")
Cs=[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
Xmatrix=dftouse[shooter].values
Yresp=dftouse['SHOT_MADE_FLAG'].values

In [43]:
def cv_optimize(clf, parameters, X, y, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    gs.fit(X, y)
    print "BEST PARAMS", gs.best_params_
    best = gs.best_estimator_
    return best

from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    if mask !=None:
        print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

In [44]:
%%time
clfsvm, Xtrain, ytrain, Xtest, ytest = do_classify(LinearSVC(loss="hinge"), {"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,shooter, u'SHOT_MADE_FLAG',1, mask=mask)

using mask


/Users/raymondcen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').